# 6.0 Transfer Learning with Pre-trained Model (CNN)

In [5]:
import pretrainedmodels
# https://github.com/Cadene/pretrained-models.pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [2]:
# Start with the lightweight model for experiment

# We choose Resnet34 for our initial transfer learning

In [3]:
model_name = 'resnet34'
backbone = pretrainedmodels.__dict__[model_name](pretrained='imagenet')

In [4]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# With torch.nn , we can acccess each of the layers in the pre-trained model
backbone.layer4

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1

### 1.0 We need to convert the color channel to black/gray (1 channel), instead of original Imagenet color channel (3 channel)

In [10]:
# original
backbone.conv1 

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [11]:
# changed
backbone.conv1 = nn.Conv2d(1,64,7,2,3, bias = False)

### 2.0 Convert the last out_features to be classes of 3 different output for this competition
#### (i.e.it has 186 classes)

In [12]:
in_features = backbone.last_linear.in_features
in_features

512

In [13]:
backbone.last_linear = nn.Linear(in_features, 186)

In [15]:
# check it has changed to 186 output features
backbone.last_linear

Linear(in_features=512, out_features=186, bias=True)

### 3.0 Test out the customized Pre-trained model

In [17]:
batches = torch.rand(6,1,137,236)

In [18]:
batches.shape

torch.Size([6, 1, 137, 236])

In [19]:
outputs = backbone(batches)

In [20]:
outputs.shape

torch.Size([6, 186])

In [21]:
# logits
outputs

tensor([[ 0.0101, -0.3235, -0.0507,  ..., -0.4847, -0.9483,  0.5342],
        [-0.4956, -0.1648,  0.9268,  ..., -0.0569, -0.2465, -0.0174],
        [-0.2799,  0.3643,  0.3467,  ..., -0.4445, -0.4600, -0.4748],
        [-0.4254, -1.0537, -0.1966,  ..., -0.4261, -0.1965, -0.4405],
        [-0.5621, -0.4185,  0.4717,  ..., -0.0215, -0.4796,  0.4409],
        [-0.0734, -0.5643,  0.3597,  ..., -0.0155, -0.3334, -0.0791]],
       grad_fn=<AddmmBackward>)

In [22]:
outputs.max()

tensor(2.0960, grad_fn=<MaxBackward1>)

In [23]:
outputs.min()

tensor(-2.7494, grad_fn=<MinBackward1>)